In [9]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import os, sys
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
%matplotlib inline

# Upload training data file

In [3]:
df = pd.read_csv("traininglabels.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 3 columns):
image_id       15244 non-null object
has_oilpalm    15244 non-null int64
score          15244 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 357.4+ KB


In [107]:
df[df.has_oilpalm == 1].count() # only 6% of the images have palm oil plantations

image_id       942
has_oilpalm    942
score          942
dtype: int64

In [113]:
df.score.sort_values() 
# some images have very small scores. Perhaps use a threshold like 0.5 and give 0.5 or less, label 0

1062     0.3887
72       0.3944
353      0.3954
1404     0.3963
270      0.3965
4163     0.3970
3032     0.3973
5330     0.3993
8508     0.3994
1942     0.3994
9956     0.3995
10327    0.3996
1969     0.4003
14359    0.4006
1532     0.4006
7699     0.4010
8853     0.4012
715      0.4018
13936    0.4020
9844     0.4026
14229    0.4032
5541     0.4035
14625    0.4037
5937     0.4042
13531    0.4042
3234     0.4043
14813    0.4045
5242     0.4045
9099     0.4045
4756     0.4047
          ...  
5496     1.0000
5495     1.0000
5494     1.0000
5493     1.0000
5466     1.0000
5467     1.0000
5468     1.0000
5469     1.0000
5470     1.0000
5471     1.0000
5472     1.0000
5473     1.0000
5474     1.0000
5477     1.0000
5478     1.0000
5464     1.0000
5479     1.0000
5481     1.0000
5482     1.0000
5483     1.0000
5484     1.0000
5485     1.0000
5486     1.0000
5487     1.0000
5488     1.0000
5489     1.0000
5490     1.0000
5491     1.0000
5480     1.0000
15243    1.0000
Name: score, Length: 152

# Read Train images and build a model

In [73]:
path = "/Users/HS/Downloads/widsdatathon2019/train_images"
imagesList = sorted(os.listdir(path))
imagesList

['img_000002017.jpg',
 'img_000002018.jpg',
 'img_000012017.jpg',
 'img_000022017.jpg',
 'img_000032018.jpg',
 'img_000042018.jpg',
 'img_000052018.jpg',
 'img_000072017.jpg',
 'img_000072018.jpg',
 'img_000082017.jpg',
 'img_000082018.jpg',
 'img_000092017.jpg',
 'img_000092018.jpg',
 'img_000102017.jpg',
 'img_000102018.jpg',
 'img_000112017.jpg',
 'img_000112018.jpg',
 'img_000122018.jpg',
 'img_000132017.jpg',
 'img_000142017.jpg',
 'img_000152017.jpg',
 'img_000152018.jpg',
 'img_000172017.jpg',
 'img_000172018.jpg',
 'img_000182018.jpg',
 'img_000192017.jpg',
 'img_000192018.jpg',
 'img_000202017.jpg',
 'img_000202018.jpg',
 'img_000212017.jpg',
 'img_000212018.jpg',
 'img_000232017.jpg',
 'img_000232018.jpg',
 'img_000242017.jpg',
 'img_000242018.jpg',
 'img_000252017.jpg',
 'img_000272017.jpg',
 'img_000272018.jpg',
 'img_000282017.jpg',
 'img_000282018.jpg',
 'img_000292018.jpg',
 'img_000302017.jpg',
 'img_000312017.jpg',
 'img_000312018.jpg',
 'img_000322017.jpg',
 'img_0003

In [74]:
# Reading images from a folder and converting to numpy array
train_image = []
for i in tqdm(range(len(imagesList))):
    img = image.load_img('/Users/HS/Downloads/widsdatathon2019/train_images/'+imagesList[i], target_size=(28,28,1))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
X

100%|██████████| 15244/15244 [00:19<00:00, 782.89it/s]


array([[[[ 0.20392157,  0.25098041,  0.10980392],
         [ 0.22352941,  0.25490198,  0.10980392],
         [ 0.05490196,  0.1254902 ,  0.03137255],
         ..., 
         [ 0.11372549,  0.18431373,  0.09019608],
         [ 0.04705882,  0.11764706,  0.03137255],
         [ 0.0627451 ,  0.12156863,  0.03921569]],

        [[ 0.15294118,  0.21960784,  0.09411765],
         [ 0.03529412,  0.10588235,  0.00392157],
         [ 0.10980392,  0.17254902,  0.07058824],
         ..., 
         [ 0.13333334,  0.18431373,  0.07450981],
         [ 0.11372549,  0.16078432,  0.07450981],
         [ 0.02745098,  0.09411765,  0.01960784]],

        [[ 0.08627451,  0.15686275,  0.0627451 ],
         [ 0.04313726,  0.11372549,  0.01176471],
         [ 0.04705882,  0.11372549,  0.03921569],
         ..., 
         [ 0.09019608,  0.15294118,  0.05098039],
         [ 0.0627451 ,  0.11372549,  0.03529412],
         [ 0.1254902 ,  0.18039216,  0.08235294]],

        ..., 
        [[ 0.06666667,  0.11372549,

In [75]:
y = df['has_oilpalm'].values
y = to_categorical(y)
y

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]], dtype=float32)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [77]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [78]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [79]:
# Build model with training data and get loss and accuracy scores
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 12195 samples, validate on 3049 samples
Epoch 1/10
12195/12195 [==============================] - 19s 2ms/step - loss: 0.2529 - acc: 0.9356 - val_loss: 0.2542 - val_acc: 0.9403
Epoch 2/10
12195/12195 [==============================] - 20s 2ms/step - loss: 0.2448 - acc: 0.9377 - val_loss: 0.2296 - val_acc: 0.9403
Epoch 3/10
12195/12195 [==============================] - 22s 2ms/step - loss: 0.2423 - acc: 0.9377 - val_loss: 0.2262 - val_acc: 0.9403
Epoch 4/10
12195/12195 [==============================] - 22s 2ms/step - loss: 0.2395 - acc: 0.9377 - val_loss: 0.2291 - val_acc: 0.9403
Epoch 5/10
12195/12195 [==============================] - 22s 2ms/step - loss: 0.2379 - acc: 0.9377 - val_loss: 0.2277 - val_acc: 0.9403
Epoch 6/10
12195/12195 [==============================] - 20s 2ms/step - loss: 0.2343 - acc: 0.9377 - val_loss: 0.2281 - val_acc: 0.9403
Epoch 7/10
12195/12195 [==============================] - 21s 2ms/step - loss: 0.2302 - acc: 0.9377 - val_loss: 0.2298 - val_acc:

# Leaderboard Test Data Prediction

In [80]:
testpath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data"
testimagesList = sorted(os.listdir(testpath))
testimagesList

['img_000022018.jpg',
 'img_000042017.jpg',
 'img_000052017.jpg',
 'img_000062017.jpg',
 'img_000062018.jpg',
 'img_000122017.jpg',
 'img_000142018.jpg',
 'img_000162018.jpg',
 'img_000182017.jpg',
 'img_000262017.jpg',
 'img_000292017.jpg',
 'img_000302018.jpg',
 'img_000342018.jpg',
 'img_000382017.jpg',
 'img_000382018.jpg',
 'img_000422017.jpg',
 'img_000422018.jpg',
 'img_000432017.jpg',
 'img_000472018.jpg',
 'img_000532017.jpg',
 'img_000542018.jpg',
 'img_000552017.jpg',
 'img_000592017.jpg',
 'img_000602018.jpg',
 'img_000622017.jpg',
 'img_000622018.jpg',
 'img_000632018.jpg',
 'img_000672018.jpg',
 'img_000682018.jpg',
 'img_000692017.jpg',
 'img_000702017.jpg',
 'img_000752017.jpg',
 'img_000762017.jpg',
 'img_000812018.jpg',
 'img_000822018.jpg',
 'img_000832018.jpg',
 'img_000842018.jpg',
 'img_000852018.jpg',
 'img_000872018.jpg',
 'img_000882017.jpg',
 'img_000882018.jpg',
 'img_000902018.jpg',
 'img_000912017.jpg',
 'img_000912018.jpg',
 'img_000922018.jpg',
 'img_0009

In [84]:
leaderboard_test_pred = pd.DataFrame(testimagesList , columns=['image_id'])
leaderboard_test_pred

,image_id
0,img_000022018.jpg
1,img_000042017.jpg
2,img_000052017.jpg
3,img_000062017.jpg
4,img_000062018.jpg
5,img_000122017.jpg
6,img_000142018.jpg
7,img_000162018.jpg
8,img_000182017.jpg
9,img_000262017.jpg


In [81]:
test_image = []
for i in tqdm(range(len(testimagesList))):
    img = image.load_img('/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data/'+testimagesList[i], target_size=(28,28,1))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)
test

100%|██████████| 4356/4356 [00:06<00:00, 628.72it/s]


array([[[[ 0.56470591,  0.48235294,  0.3764706 ],
         [ 0.64705884,  0.5529412 ,  0.45882353],
         [ 0.7764706 ,  0.66666669,  0.57254905],
         ..., 
         [ 0.17254902,  0.18039216,  0.12941177],
         [ 0.17647059,  0.19215687,  0.13725491],
         [ 0.16470589,  0.17254902,  0.12156863]],

        [[ 0.49803922,  0.41176471,  0.31764707],
         [ 0.59215689,  0.51372552,  0.41960785],
         [ 0.81176472,  0.69411767,  0.59215689],
         ..., 
         [ 0.16470589,  0.17254902,  0.12156863],
         [ 0.16470589,  0.17647059,  0.14117648],
         [ 0.15686275,  0.16862746,  0.1254902 ]],

        [[ 0.84313726,  0.78823531,  0.65098041],
         [ 0.56862748,  0.49411765,  0.3764706 ],
         [ 0.98039216,  0.90196079,  0.80784315],
         ..., 
         [ 0.14117648,  0.16078432,  0.13333334],
         [ 0.16078432,  0.16862746,  0.11764706],
         [ 0.16862746,  0.18431373,  0.1254902 ]],

        ..., 
        [[ 0.9254902 ,  0.81176472,

In [86]:
# making predictions
prediction = model.predict_classes(test)

In [87]:
leaderboard_test_pred['has_oilpalm'] = prediction

In [115]:
leaderboard_test_pred.has_oilpalm.unique()

array([0])

In [89]:
leaderboard_test_pred[leaderboard_test_pred.has_oilpalm == 1]

,image_id,has_oilpalm


# Leaderboard Holdback Data Prediction

In [92]:
holdpath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data"
holdimagesList = sorted(os.listdir(holdpath))
holdimagesList

['img_000012018.jpg',
 'img_000032017.jpg',
 'img_000132018.jpg',
 'img_000162017.jpg',
 'img_000222017.jpg',
 'img_000222018.jpg',
 'img_000252018.jpg',
 'img_000262018.jpg',
 'img_000432018.jpg',
 'img_000482017.jpg',
 'img_000482018.jpg',
 'img_000502017.jpg',
 'img_000572017.jpg',
 'img_000612018.jpg',
 'img_000642017.jpg',
 'img_000642018.jpg',
 'img_000652018.jpg',
 'img_000682017.jpg',
 'img_000722017.jpg',
 'img_000742018.jpg',
 'img_000772018.jpg',
 'img_000842017.jpg',
 'img_000922017.jpg',
 'img_001022017.jpg',
 'img_001092018.jpg',
 'img_001102018.jpg',
 'img_001162017.jpg',
 'img_001172017.jpg',
 'img_001242018.jpg',
 'img_001282017.jpg',
 'img_001282018.jpg',
 'img_001352017.jpg',
 'img_001502017.jpg',
 'img_001612017.jpg',
 'img_001622018.jpg',
 'img_001672018.jpg',
 'img_001822018.jpg',
 'img_001832017.jpg',
 'img_001882017.jpg',
 'img_001962018.jpg',
 'img_002062017.jpg',
 'img_002112018.jpg',
 'img_002182017.jpg',
 'img_002202018.jpg',
 'img_002242017.jpg',
 'img_0023

In [93]:
leaderboard_hold_pred = pd.DataFrame(holdimagesList , columns=['image_id'])
leaderboard_hold_pred

,image_id
0,img_000012018.jpg
1,img_000032017.jpg
2,img_000132018.jpg
3,img_000162017.jpg
4,img_000222017.jpg
5,img_000222018.jpg
6,img_000252018.jpg
7,img_000262018.jpg
8,img_000432018.jpg
9,img_000482017.jpg


In [94]:
hold_image = []
for i in tqdm(range(len(holdimagesList))):
    img = image.load_img('/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data/'+holdimagesList[i], target_size=(28,28,1))
    img = image.img_to_array(img)
    img = img/255
    hold_image.append(img)
hold = np.array(hold_image)
hold

100%|██████████| 2178/2178 [00:05<00:00, 435.06it/s]


array([[[[ 0.14117648,  0.20784314,  0.08235294],
         [ 0.12941177,  0.18431373,  0.05490196],
         [ 0.12941177,  0.18039216,  0.0627451 ],
         ..., 
         [ 0.14509805,  0.21960784,  0.10196079],
         [ 0.10980392,  0.1882353 ,  0.05098039],
         [ 0.12156863,  0.19607843,  0.07843138]],

        [[ 0.14509805,  0.21960784,  0.09411765],
         [ 0.10980392,  0.18431373,  0.05882353],
         [ 0.14901961,  0.21568628,  0.08235294],
         ..., 
         [ 0.09411765,  0.17254902,  0.0627451 ],
         [ 0.13725491,  0.21176471,  0.08627451],
         [ 0.11372549,  0.1882353 ,  0.07058824]],

        [[ 0.15686275,  0.23529412,  0.09019608],
         [ 0.16470589,  0.24705882,  0.09411765],
         [ 0.16470589,  0.23921569,  0.11372549],
         ..., 
         [ 0.09411765,  0.16862746,  0.05882353],
         [ 0.09411765,  0.16862746,  0.05098039],
         [ 0.1254902 ,  0.2       ,  0.08235294]],

        ..., 
        [[ 0.15294118,  0.20784314,

In [95]:
# making predictions
prediction = model.predict_classes(hold)

In [97]:
leaderboard_hold_pred['has_oilpalm'] = prediction

In [114]:
leaderboard_hold_pred.has_oilpalm.unique()

array([0])

In [98]:
leaderboard_test_pred[leaderboard_test_pred.has_oilpalm == 1]

,image_id,has_oilpalm


# Building submission file

In [119]:
leaderboard_test_pred.shape

(4356, 2)

In [120]:
leaderboard_hold_pred.shape

(2178, 2)

In [129]:
finalSubmission = leaderboard_test_pred.append(leaderboard_hold_pred)
finalSubmission.shape

(6534, 2)

In [130]:
finalSubmission.head()

,image_id,has_oilpalm
0,img_000022018.jpg,0
1,img_000042017.jpg,0
2,img_000052017.jpg,0
3,img_000062017.jpg,0
4,img_000062018.jpg,0


In [131]:
finalSubmission.to_csv("finalSubmission.csv",index=False)